# 12. 얼굴 검출 (Face Detection)

Viola-Jones 알고리즘의 핵심 개념을 이해합니다.

## 학습 목표
- Haar-like 특징 이해
- 적분 이미지(Integral Image) 개념
- Cascade 분류기 구조

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. Haar-like 특징

Haar 특징은 밝은 영역과 어두운 영역의 차이를 계산합니다.

In [ ]:
# Haar 특징 시각화
fig, axes = plt.subplots(1, 4, figsize=(12, 3))

# 수평 에지
haar_h = torch.ones(4, 4)
haar_h[:2, :] = -1
axes[0].imshow(haar_h, cmap='RdBu', vmin=-1, vmax=1)
axes[0].set_title('Horizontal Edge')
axes[0].axis('off')

# 수직 에지
haar_v = torch.ones(4, 4)
haar_v[:, :2] = -1
axes[1].imshow(haar_v, cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title('Vertical Edge')
axes[1].axis('off')

# 수평 선
haar_line = torch.ones(6, 4)
haar_line[2:4, :] = -1
axes[2].imshow(haar_line, cmap='RdBu', vmin=-1, vmax=1)
axes[2].set_title('Horizontal Line')
axes[2].axis('off')

# 사각형 (눈 영역 등)
haar_rect = torch.ones(4, 4)
haar_rect[1:3, 1:3] = -1
axes[3].imshow(haar_rect, cmap='RdBu', vmin=-1, vmax=1)
axes[3].set_title('Center Rectangle')
axes[3].axis('off')

plt.suptitle('Haar-like Features (White = +1, Blue = -1)', fontsize=12)
plt.tight_layout()
plt.show()

## 2. 적분 이미지 (Integral Image)

적분 이미지를 사용하면 임의 크기의 영역 합을 O(1)에 계산할 수 있습니다.

In [ ]:
def compute_integral_image(img):
    """적분 이미지 계산"""
    return torch.cumsum(torch.cumsum(img, dim=0), dim=1)

def region_sum(integral, x1, y1, x2, y2):
    """적분 이미지로 영역 합 계산 (O(1))"""
    total = integral[y2, x2]
    if x1 > 0:
        total -= integral[y2, x1-1]
    if y1 > 0:
        total -= integral[y1-1, x2]
    if x1 > 0 and y1 > 0:
        total += integral[y1-1, x1-1]
    return total

# 예시
img = torch.arange(1, 17).reshape(4, 4).float()
integral = compute_integral_image(img)

print("Original Image:")
print(img)
print("\nIntegral Image:")
print(integral)

# 영역 합 테스트
print(f"\nSum of region (1,1) to (2,2): {region_sum(integral, 1, 1, 2, 2)}")
print(f"Direct sum: {img[1:3, 1:3].sum()}")

## 3. Haar 특징 계산

In [ ]:
def compute_haar_feature(integral, x, y, w, h, feature_type='horizontal_edge'):
    """Haar 특징 계산"""
    if feature_type == 'horizontal_edge':
        # 상단 - 하단
        top = region_sum(integral, x, y, x+w-1, y+h//2-1)
        bottom = region_sum(integral, x, y+h//2, x+w-1, y+h-1)
        return bottom - top
    
    elif feature_type == 'vertical_edge':
        # 좌측 - 우측
        left = region_sum(integral, x, y, x+w//2-1, y+h-1)
        right = region_sum(integral, x+w//2, y, x+w-1, y+h-1)
        return right - left
    
    return 0

# 테스트 이미지
test_img = torch.zeros(8, 8)
test_img[:4, :] = 1  # 상단 밝음
test_integral = compute_integral_image(test_img)

feat = compute_haar_feature(test_integral, 0, 0, 8, 8, 'horizontal_edge')
print(f"Horizontal edge feature: {feat}")

## 4. 얼굴의 Haar 특징

얼굴에는 특징적인 밝기 패턴이 있습니다:
- 눈 영역은 볼보다 어둡다
- 코의 중심 라인은 양 옆보다 밝다

In [ ]:
# 단순화된 얼굴 패턴 생성
face = torch.ones(24, 24) * 180  # 피부색 (밝음)

# 눈 (어두움)
face[6:10, 4:8] = 50
face[6:10, 16:20] = 50

# 코 (중간)
face[10:18, 10:14] = 120

# 눈 아래 영역 (밝음)
face[10:14, 4:8] = 200
face[10:14, 16:20] = 200

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 원본
axes[0].imshow(face, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Simplified Face Pattern')
axes[0].axis('off')

# 눈 영역 특징
mask1 = torch.zeros_like(face)
mask1[6:10, 4:8] = -1  # 눈 (어두움)
mask1[10:14, 4:8] = 1  # 눈 아래 (밝음)
axes[1].imshow(mask1, cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title('Eye Region Feature')
axes[1].axis('off')

# 양 눈 특징
mask2 = torch.zeros_like(face)
mask2[6:10, 4:20] = -1  # 눈 라인 (어두움)
mask2[10:14, 4:20] = 1  # 볼 라인 (밝음)
axes[2].imshow(mask2, cmap='RdBu', vmin=-1, vmax=1)
axes[2].set_title('Eye Line Feature')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 특징 값 계산
integral_face = compute_integral_image(face)
print("Haar Features:")
print(f"  Eye-cheek horizontal: {compute_haar_feature(integral_face, 4, 6, 4, 8, 'horizontal_edge'):.1f}")

## 5. Cascade 분류기 개념

Viola-Jones는 캐스케이드 구조로 빠른 거부(rejection)를 수행합니다.

In [ ]:
# 캐스케이드 개념 시각화
stages = ['Stage 1\n(2 features)', 'Stage 2\n(10 features)', 'Stage 3\n(25 features)', '...', 'Final\n(~6000 features)']
rejection_rates = [0.5, 0.8, 0.9, 0.95, 0.99]

fig, ax = plt.subplots(figsize=(12, 4))

# 각 스테이지 박스
for i, (stage, rate) in enumerate(zip(stages, rejection_rates)):
    rect = plt.Rectangle((i*2, 0), 1.5, 1, fill=True, 
                          facecolor=plt.cm.Blues(rate), edgecolor='black')
    ax.add_patch(rect)
    ax.text(i*2 + 0.75, 0.5, stage, ha='center', va='center', fontsize=10)
    
    if i < len(stages) - 1:
        ax.annotate('', xy=(i*2+2, 0.5), xytext=(i*2+1.5, 0.5),
                   arrowprops=dict(arrowstyle='->', color='green'))
        ax.annotate('Non-face', xy=(i*2+0.75, -0.3), xytext=(i*2+0.75, 0),
                   arrowprops=dict(arrowstyle='->', color='red'),
                   ha='center', fontsize=8, color='red')

ax.set_xlim(-0.5, 11)
ax.set_ylim(-0.8, 1.5)
ax.axis('off')
ax.set_title('Cascade Classifier Structure', fontsize=14, pad=20)

# 범례 추가
ax.text(5, 1.3, 'Pass → Next Stage (Green)', fontsize=10, color='green')
ax.text(5, 1.1, 'Fail → Reject as Non-face (Red)', fontsize=10, color='red')

plt.tight_layout()
plt.show()

In [ ]:
# 캐스케이드 효율성 시뮬레이션
n_windows = 100000  # 스캔할 윈도우 수
face_ratio = 0.001  # 실제 얼굴 비율

# 각 스테이지 특성
stage_features = [2, 10, 25, 50, 100]  # 특징 수
stage_reject_rate = [0.5, 0.7, 0.8, 0.9, 0.95]  # 비얼굴 거부율

remaining = n_windows
total_features_evaluated = 0

print(f"Initial windows: {n_windows:,}")
print(f"Expected faces: {int(n_windows * face_ratio)}")
print("\nCascade Progression:")

for i, (n_feat, reject) in enumerate(zip(stage_features, stage_reject_rate)):
    features_this_stage = remaining * n_feat
    total_features_evaluated += features_this_stage
    
    rejected = int(remaining * reject * (1 - face_ratio))  # 비얼굴만 거부
    remaining -= rejected
    
    print(f"  Stage {i+1}: {remaining:,} windows remain (evaluated {n_feat} features each)")

print(f"\nTotal feature evaluations: {total_features_evaluated:,.0f}")
print(f"Without cascade (all 6000 features): {n_windows * 6000:,}")
print(f"Speedup: {(n_windows * 6000) / total_features_evaluated:.1f}x")

## 요약

1. **Haar-like 특징**: 밝은/어두운 영역 차이로 에지, 선 검출
2. **적분 이미지**: O(1)에 영역 합 계산
3. **AdaBoost**: 수천 개 특징 중 중요한 것 선택
4. **Cascade**: 빠른 거부로 실시간 처리 가능

Viola-Jones 알고리즘은 실시간 얼굴 검출의 기초가 되었습니다.

다음: **희소 코딩 (Sparse Coding)**